## Convert https://www.braceletbook.com/ pixel art to png

This notebook will take a downloaded image from braceletbook.com and convert the image to its proper png scale for pixel art.

In [16]:
import cv2
import numpy as np
import time

def create_color_grid(image_path, output_path):
    print("Starting image processing...")
    start_time = time.time()
    
    # Read the image
    img = cv2.imread(image_path)
    if img is None:
        raise Exception("Could not read the image")
    
    # Convert to grayscale for box detection
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 131, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Store box centers and their colors
    box_data = []
    
    for contour in contours:
        peri = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * peri, True)
        
        if len(approx) == 4:
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = float(w)/h
            if 0.2 < aspect_ratio < 5:
                # Get center point
                center_x = int(x + w/2)
                center_y = int(y + h/2)
                
                # Sample color at center point
                color = img[center_y, center_x]
                
                box_data.append({
                    'center': (center_x, center_y),
                    'color': color
                })
    
    # Find grid dimensions
    tolerance = 5
    centers = np.array([box['center'] for box in box_data])
    unique_x = np.unique(np.round(centers[:, 0] / tolerance)) * tolerance
    unique_y = np.unique(np.round(centers[:, 1] / tolerance)) * tolerance
    
    num_rows = len(unique_y)
    num_cols = len(unique_x)
    
    print(f"Detected grid: {num_rows} rows x {num_cols} columns")
    
    # Create new image (single pixel per box)
    new_img = np.zeros((num_rows, num_cols, 3), dtype=np.uint8)
    
    # Map box centers to grid positions
    for box in box_data:
        x, y = box['center']
        # Find nearest grid position
        col = np.argmin(np.abs(unique_x - x))
        row = np.argmin(np.abs(unique_y - y))
        new_img[row, col] = box['color']
    
    # Save the result
    cv2.imwrite(output_path, new_img)
    
    print(f"Total time: {time.time() - start_time:.2f} seconds")
    print(f"Output saved to: {output_path}")
    
    return new_img

# Use the function
input_path = 'image_input.png'
output_path = 'image_output.png'
try:
    result = create_color_grid(input_path, output_path)
except Exception as e:
    print(f"An error occurred: {e}")

Starting image processing...
Detected grid: 8 rows x 17 columns
Total time: 0.01 seconds
Output saved to: image_output.png
